# 10장. 특성 선택을 사용한 차원 축소

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-python-cookbook-2nd/blob/main/ch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [1]:
import numpy as np
import pandas as pd
import sklearn

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('sklearn', sklearn.__version__)

numpy 1.26.4
pandas 2.2.2
sklearn 1.6.0


## 10.1 분산을 기준으로 수치 특성 선택하기

In [2]:
# 라이브러리를 임포트합니다.
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# 예제 데이터를 로드합니다.
iris = datasets.load_iris()

# 특성과 타깃을 만듭니다.
features = iris.data
target = iris.target

# 기준값을 만듭니다.
thresholder = VarianceThreshold(threshold=.5)

# 기준값보다 높은 특성을 선택합니다.
features_high_variance = thresholder.fit_transform(features)

# 선택한 특성을 확인합니다.
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [3]:
# 분산을 확인합니다.
thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [4]:
# 라이브러리를 임포트합니다.
from sklearn.preprocessing import StandardScaler

# 특성 행렬을 표준화합니다.
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# 각 특성의 분산을 계산합니다.
selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## 10.2 분산을 기준으로 이진 특성 선택하기

In [5]:
# 라이브러리를 임포트합니다.
from sklearn.feature_selection import VarianceThreshold

# 예제 특성 행렬을 만듭니다.
# 특성 0: 80%가 클래스 0
# 특성 1: 80%가 클래스 1
# 특성 2: 60%가 클래스 0, 40%는 클래스 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

### 붙임

In [6]:
thresholder.variances_

array([0.16, 0.16, 0.24])

In [7]:
import numpy as np
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

## 10.3 상관관계가 큰 특성 다루기

In [8]:
# 라이브러리를 임포트합니다.
import pandas as pd
import numpy as np

# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

# 특성 행렬을 DataFrame으로 변환합니다.
dataframe = pd.DataFrame(features)

# 상관관계 행렬을 만듭니다.
corr_matrix = dataframe.corr().abs()

# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택합니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
                          k=1).astype(bool))

# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# 특성을 삭제합니다.
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [9]:
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [10]:
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


### 붙임

In [11]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [12]:
np.triu(np.ones((4, 4)), k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [13]:
np.tril(np.ones((4, 4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

## 10.4 분류 작업에 관련 없는 특성 삭제하기

In [14]:
# 라이브러리를 임포트합니다.
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# 데이터를 로드합니다.
iris = load_iris()
features = iris.data
target = iris.target

# 범주형 데이터를 정수형으로 변환합니다.
features = features.astype(int)

# 카이제곱 통계값이 가장 큰 특성 두 개를 선택합니다.
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [15]:
# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [16]:
# 라이브러리를 임포트합니다.
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 선택합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 3


### 붙임

카이제곱 계산

In [17]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [18]:
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [19]:
expected = features.sum(axis=0) / 3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [20]:
np.sum((observed - expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [21]:
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

ANOVA 계산

In [22]:
total_mean = np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [23]:
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [24]:
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

In [25]:
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [26]:
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [27]:
fvalue_selector.scores_

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

## 10.5 재귀적 특성 제거

In [28]:
# 라이브러리를 임포트합니다.
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 ],
       [-1.07500204,  2.56148527],
       [ 1.37940721, -1.77039484],
       ...,
       [-0.80331656, -1.60648007],
       [ 0.39508844, -1.34564911],
       [-0.55383035,  0.82880112]])

In [29]:
# 최선의 특성 개수
rfecv.n_features_

2

In [30]:
# 선택된 특성이 표시된 불리언 마스크
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [31]:
# 특성의 순위: 최고(1)에서 최악(96)까지
rfecv.ranking_

array([43, 56, 91, 38, 54,  1, 18, 89, 16, 28, 51, 60,  9, 62, 15, 49, 34,
       47, 21, 11, 29, 39, 88,  5, 23, 66, 22, 30, 98, 24, 79, 73, 48, 74,
       20, 33, 99,  3, 70,  1, 19, 75, 63, 57, 58, 80, 45, 41, 25, 55, 96,
       93, 65, 77, 46, 12, 37, 90, 61, 82, 31, 52, 78, 32, 64, 68, 72, 26,
        4, 69, 53,  8, 44, 92, 50, 10, 86, 36, 40, 87, 71, 14, 35, 27, 17,
        7, 94,  6, 84, 95, 42, 67,  2, 83, 13, 97, 59, 76, 81, 85])

### 붙임

In [32]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)
rfe.fit(features, target)
rfe.transform(features)

array([[ 0.00850799,  0.7031277 ,  0.39544554],
       [-1.07500204,  2.56148527, -0.66998145],
       [ 1.37940721, -1.77039484, -0.17881197],
       ...,
       [-0.80331656, -1.60648007,  0.01579457],
       [ 0.39508844, -1.34564911, -1.21773707],
       [-0.55383035,  0.82880112,  0.79632039]])

In [33]:
np.all(rfe.support_ == rfecv.support_)

False